# Revista da Associação Brasileira de Custos – ABCustos

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(query, url_search_text):
    url_search = url_search_text.replace('__query__', query)
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in div_results:
        try:
            a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
url_search_text = 'https://revista.abcustos.org.br/abcustos/search/search?query=__query__&dateFromYear=2012&dateFromMonth=01&dateFromDay=1&dateToYear=2021&dateToMonth=12&dateToDay=31&authors='

listar_links_pagina('contabilidade+ambiental', url_search_text)


['https://revista.abcustos.org.br/abcustos/article/view/497',
 'https://revista.abcustos.org.br/abcustos/article/view/506',
 'https://revista.abcustos.org.br/abcustos/article/view/548',
 'https://revista.abcustos.org.br/abcustos/article/view/212']

In [5]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao

In [6]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [7]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [8]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date}

In [9]:
def captura_resumo(soup):
    try:
        abstract = soup.find('section', {'class': 'item abstract'}).text
        abstract = abstract.replace('Resumo', '')
        abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [10]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('section', {'class': 'item keywords'})
        kw = kw_tag.find('span', {'class': 'value'}).text.replace(',', '.').replace(';', '.')
        kw = ' '.join(kw.split())
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [11]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [12]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [limpar_afiliacao(a.text) for a in afiliacao]
        
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [13]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista da Associação Brasileira de Custos – ABCustos'}
    url_search_text = 'https://revista.abcustos.org.br/abcustos/search/search?query=__query__&dateFromYear=2012&dateFromMonth=01&dateFromDay=1&dateToYear=2021&dateToMonth=12&dateToDay=31&authors='
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links_pagina(query, url_search_text)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [14]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-05-29,ANÁLISE DE CUSTOS E DO IMPACTO AMBIENTAL NO DE...,As práticas ambientais têm se intensificado de...,Viabilidade Financeira. Desenvolvimento susten...,3,Ralf Creutzberg,Mara Juliana Ferrari,Emanuele Engelage,,,,UNIDADI,Centro Universitário do Alto Vale do Itajaí,Universidade Federal de Santa Catarina,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
1,2019-01-25,CUSTO DE PRODUÇÃO AGRÍCOLA: UMA ANÁLISE DO CUL...,A atividade de agricultura consiste em importa...,Atividade Rural. Cultivo da Soja. Custo da Ati...,2,Yvelise Piccinin,Marivane Vestena Rossato,,,,,Universidade Federal de Santa Maria,Universidade Federal de Santa Maria,,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
2,2020-05-04,ANÁLISE DA SUSTENTABILIDADE EMPRESARIAL NAS CO...,A presente pesquisa objetiva avaliar o desempe...,Sustentabilidade Empresarial. Indicadores de D...,2,Fernanda Nunes Dantas Cascudo Rodrigues,José Ribamar Marques de Carvalho,,,,,Mestra em Ciências Contábeis PPGCONT UFRN,Universidade Federal de Campina Grande,,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...


In [15]:
df.to_csv('../data/ABCUSTOS.csv', index=False)

In [16]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-05-29,ANÁLISE DE CUSTOS E DO IMPACTO AMBIENTAL NO DE...,As práticas ambientais têm se intensificado de...,Viabilidade Financeira. Desenvolvimento susten...,3,Ralf Creutzberg,Mara Juliana Ferrari,Emanuele Engelage,,,,UNIDADI,Centro Universitário do Alto Vale do Itajaí,Universidade Federal de Santa Catarina,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
1,2019-01-25,CUSTO DE PRODUÇÃO AGRÍCOLA: UMA ANÁLISE DO CUL...,A atividade de agricultura consiste em importa...,Atividade Rural. Cultivo da Soja. Custo da Ati...,2,Yvelise Piccinin,Marivane Vestena Rossato,,,,,Universidade Federal de Santa Maria,Universidade Federal de Santa Maria,,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
2,2020-05-04,ANÁLISE DA SUSTENTABILIDADE EMPRESARIAL NAS CO...,A presente pesquisa objetiva avaliar o desempe...,Sustentabilidade Empresarial. Indicadores de D...,2,Fernanda Nunes Dantas Cascudo Rodrigues,José Ribamar Marques de Carvalho,,,,,Mestra em Ciências Contábeis PPGCONT UFRN,Universidade Federal de Campina Grande,,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
3,2013-02-15,O BALANCED SCORECARD E A GESTÃO AMBIENTAL: UM ...,Os aspectos sociais e ambientais são reconheci...,Balanced Scorecard. Contabilidade de Gestão. G...,3,Patricia Rodrigues Quesado,Lúcia Lima Rodrigues,Beatriz Aibar Guzmán,,,,,,,,,,https://revista.abcustos.org.br/abcustos/artic...,Revista da Associação Brasileira de Custos – A...
